<a href="https://colab.research.google.com/github/gulfarooqi/FraudDetection/blob/master/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from scipy import stats


train_id = pd.read_csv('train_identity.csv')
train_trans = pd.read_csv('train_transaction.csv')

In [39]:
print('Number of elements in the sheet = ', train_id.size) # Total number of boxes in the Sheet
print('(rows, columns) =',train_id.shape) # Number of Rows and Columns in the DataFrame
print(train_trans.shape)

print(train_id.head())
print(train_trans.head())

Number of elements in the sheet =  5913553
(rows, columns) = (144233, 41)
(590540, 394)
   TransactionID  id_01  ...  DeviceType                     DeviceInfo
0        2987004    0.0  ...      mobile  SAMSUNG SM-G892A Build/NRD90M
1        2987008   -5.0  ...      mobile                     iOS Device
2        2987010   -5.0  ...     desktop                        Windows
3        2987011   -5.0  ...     desktop                            NaN
4        2987016    0.0  ...     desktop                          MacOS

[5 rows x 41 columns]
   TransactionID  isFraud  TransactionDT  ...  V337 V338  V339
0        2987000        0          86400  ...   NaN  NaN   NaN
1        2987001        0          86401  ...   NaN  NaN   NaN
2        2987002        0          86469  ...   NaN  NaN   NaN
3        2987003        0          86499  ...   NaN  NaN   NaN
4        2987004        0          86506  ...   0.0  0.0   0.0

[5 rows x 394 columns]


In [40]:
# Merge the two datasheets into one
df = pd.merge(train_trans, train_id, on='TransactionID', how='left') 
l = len(df.count(1)) # length of the index in df

df = df.dropna(1, thresh=l*0.35) # keep only the rows with at least 90% of the size of the total rows to be non-NA values

fraud = df.loc[df['isFraud'] == 1]
non_frauds = df.loc[df['isFraud'] == 0]

print(df.shape)
print(len(fraud))
print(len(non_frauds))

(590540, 226)
20663
569877


In [0]:
# Fill NaNs

df = df.fillna(-999)

In [0]:
# Show unique values in each column

i=4
columns = df.columns
print('The column %s has %d unique values' % (columns[i], df[columns[i]].nunique()))
# print(df[columns[i]].nunique() , type(df[columns[i]].nunique()))
i =0
for column in df:
  print('The column %s has %s unique values' % (df.columns[i], str(df[columns[i]].nunique())))
  i = i + 1
  

In [42]:
# Show categorical columns
cats = df[df.select_dtypes(include=['object']).columns]
cat_col = cats.columns
print(cat_col)


Index(['ProductCD', 'card4', 'card6', 'P_emaildomain', 'M1', 'M2', 'M3', 'M4',
       'M5', 'M6', 'M7', 'M8', 'M9'],
      dtype='object')


In [43]:
# Label encoding for categorical variables

from sklearn import preprocessing


for f in df.columns:
    if df[f].dtype=='object': 
        le = preprocessing.LabelEncoder()
        le.fit(list(df[f].values))
        df[f] = le.transform(list(df[f].values))


print(df.head())


   TransactionID  isFraud  TransactionDT  ...  V319  V320  V321
0        2987000        0          86400  ...   0.0   0.0   0.0
1        2987001        0          86401  ...   0.0   0.0   0.0
2        2987002        0          86469  ...   0.0   0.0   0.0
3        2987003        0          86499  ...   0.0   0.0   0.0
4        2987004        0          86506  ...   0.0   0.0   0.0

[5 rows x 226 columns]


In [0]:
from sklearn.model_selection import train_test_split

# split x to features, y as the label
x = df.drop(['isFraud'], axis= 1)
y = df['isFraud']

# split data for testing and evaluation
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.35)

In [45]:
clf = LogisticRegression(C=1e6)
clf.fit(X_train, y_train)
print("Score: ", clf.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Score:  0.9652714948545883
